In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import datetime

In [22]:
file_paths = ["../../data/Stats_csv/Processed_Defender_data.csv", 
"../../data/Stats_csv/Processed_Forward_data.csv", 
"../../data/Stats_csv/Processed_Goalkeeper_data.csv", 
"../../data/Stats_csv/Processed_Midfielder_data.csv"]
dfs = []
for file in file_paths:
    dfs.append(pd.read_csv(file))
common_columns = set(dfs[0].columns) 
for df in dfs[1:]:
    common_columns &= set(df.columns) 

common_columns = list(common_columns)
merged_df = []
for df in dfs:
    merged_df.append(df[common_columns])

merged_df = pd.concat(merged_df,ignore_index=True)
merged_df.columns


Index(['Goals with right foot', 'Crosses', 'Height', 'Yellow cards',
       'Blocked shots', 'Assists', 'Nationality', 'Offsides',
       'Headed Clearance', 'Appearances', 'Position', 'Interceptions',
       'Headed goals', 'Goals with left foot', 'Passes', 'Clearances',
       'Hit woodwork', 'Tackles', 'Red cards', 'Date of Birth', 'Name',
       'Goals', 'Losses', 'Club', 'Big Chances Created', 'Fouls', 'Wins',
       'Passes per match'],
      dtype='object')

In [23]:
merged_df

,Goals with right foot,Crosses,Height,Yellow cards,Blocked shots,Assists,Nationality,Offsides,Headed Clearance,Appearances,...,Red cards,Date of Birth,Name,Goals,Losses,Club,Big Chances Created,Fouls,Wins,Passes per match
0,0,3,188.0,5,3,0,England,0,26,13,...,0,30/01/2002,Taylor Harwood-Bellis,1,10,Southampton,1,13,1,68.54
1,0,52,179.0,3,1,1,England,0,10,13,...,0,08/09/2004,Lewis Hall,0,4,Newcastle United,3,5,5,49.85
2,1,2,182.0,2,2,1,England,0,49,13,...,0,13/07/2000,Marc Guéhi,2,6,Crystal Palace,2,6,1,52.38
3,0,6,190.0,1,3,0,Brazil,0,15,13,...,0,19/12/1997,Gabriel Magalhães,3,2,Arsenal,1,9,6,54.23
4,1,18,180.0,4,4,2,Algeria,1,13,12,...,0,06/06/2001,Rayan Aït-Nouri,3,7,Wolverhampton Wanderers,1,17,2,35.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,0,5,180.0,1,1,0,Ukraine,0,2,8,...,0,01/03/2004,Yehor Yarmoliuk,0,4,Brentford,0,2,2,13.13
612,3,3,176.0,1,3,1,DR Congo,1,2,9,...,0,03/09/1996,Yoane Wissa,7,4,Brentford,0,6,4,16.67
613,0,60,176.0,3,6,4,Belgium,0,5,12,...,0,07/05/1997,Youri Tielemans,0,3,Aston Villa,7,19,5,61.75
614,0,0,181.0,0,0,0,Turkey,0,0,4,...,0,10/01/2006,Yunus Konak,0,1,Brentford,0,2,2,3.75


In [19]:

# chuyển vị trí, quốc gia, clb thành số
for col in ['Position', 'Nationality', 'Club']:
    merged_df[col] = LabelEncoder().fit_transform(merged_df[col])


In [20]:
# Chuyển đổi với dayfirst=True
merged_df['Date of Birth'] = pd.to_datetime(merged_df['Date of Birth'], dayfirst=True, errors='coerce')

# Loại bỏ giá trị thiếu (nếu có) sau chuyển đổi
merged_df = merged_df.dropna(subset=['Date of Birth'])

# Tính tuổi
merged_df['Age'] = datetime.datetime.now().year - merged_df['Date of Birth'].dt.year

# Loại bỏ cột 'Date of Birth'
data = merged_df.drop('Date of Birth', axis=1)
data

,Goals with right foot,Crosses,Height,Yellow cards,Blocked shots,Assists,Nationality,Offsides,Headed Clearance,Appearances,...,Red cards,Name,Goals,Losses,Club,Big Chances Created,Fouls,Wins,Passes per match,Age
0,0,3,188.0,5,3,0,18,0,26,13,...,0,Taylor Harwood-Bellis,1,10,29,1,13,1,68.54,22
1,0,52,179.0,3,1,1,18,0,10,13,...,0,Lewis Hall,0,4,23,3,5,5,49.85,20
2,1,2,182.0,2,2,1,18,0,49,13,...,0,Marc Guéhi,2,6,11,2,6,1,52.38,24
3,0,6,190.0,1,3,0,6,0,15,13,...,0,Gabriel Magalhães,3,2,0,1,9,6,54.23,27
4,1,18,180.0,4,4,2,1,1,13,12,...,0,Rayan Aït-Nouri,3,7,34,1,17,2,35.33,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611,0,5,180.0,1,1,0,52,0,2,8,...,0,Yehor Yarmoliuk,0,4,7,0,2,2,13.13,20
612,3,3,176.0,1,3,1,14,1,2,9,...,0,Yoane Wissa,7,4,7,0,6,4,16.67,28
613,0,60,176.0,3,6,4,5,0,5,12,...,0,Youri Tielemans,0,3,2,7,19,5,61.75,27
614,0,0,181.0,0,0,0,51,0,0,4,...,0,Yunus Konak,0,1,7,0,2,2,3.75,18
